In [16]:
sc.stop()
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark.sql.types import *
nPartitions=4
conf = (SparkConf()
         .setMaster("local["+str(nPartitions)+"]")
         .setAppName("Ranking")
#         .set("spark.executor.memory", "5g")
#         .set("spark.driver.memory", "5g")
#         .set("spark.python.worker.memory", "5g")
       )
sc = SparkContext(conf=conf)
sqlContext = SQLContext(sc)

In [2]:
import json
import time

In [20]:
sc.__class__

pyspark.context.SparkContext

In [3]:
indexControl=27

##### 

In [22]:
varMat.__class__

pyspark.rdd.RDD

In [21]:
filename="genoMatTolControlRare.txt"
#filename="geno10m30.txt"
varMat = sc.textFile(filename)
varMat.take(2)

[u' 1:109373320:A:G 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0',
 u' 1:150524951:G:A 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0']

In [9]:
varMat.count()

34940

In [15]:
df = SQLContext(sc.textFile(filename))

AttributeError: 'RDD' object has no attribute '_jsc'

In [2]:
varMat.write.parquet("genomat.parquet")

AttributeError: 'RDD' object has no attribute 'write'

#Common functions

In [6]:
def splitValues(variantData):
    variantData=variantData.split(" ")
    variantGene=variantData.pop(0)
    variantLocus=variantData.pop(0)
    
    return (variantGene,(variantLocus,variantData))

def makePairParts(k,v,nbPart):
    result=[]
    for i in range(0,nbPart):
        result.append(((k,i),v))
        
    return [(str(sorted([k,i])),(v)) for i in range(0,nbPart)]

def f(splitIndex ,v): 
    return [(splitIndex,list(v))]

#Variant

##De Novo

In [ ]:
def scoreDeNovo(variantData):
    variantData=variantData.split(" ")
    variantGene=variantData.pop(0)
    variantLocus=variantData.pop(0)
    
    score=0
    listTrios=[]
    checkValid=0
    nTrios=len(variantData)/3
    for i in range(0,nTrios):
        checkValid=checkValid+int(variantData[i*3+1])+int(variantData[i*3+2])
    
    if checkValid==0:                              
        for i in range(0,nTrios):
            scoreTrio=int(variantData[(i*3)]=="1" and variantData[i*3+1]=="0" and variantData[i*3+2]=="0")
            if scoreTrio>0:
                listTrios.append(i)
                score=score+scoreTrio
            
    return ((variantGene,variantLocus),score,listTrios,variantData)

In [ ]:
#start_time = time.time()
#scores=sc.textFile(filename).repartition(nPartitions).map(scoreDeNovo).filter(lambda (k,v1,v2,v3): v1>0).takeOrdered(100000, key=lambda (k,v1,v2,v3): -v1)
#runtime=time.time() - start_time
#print(runtime)

In [ ]:
#scores=["TriosDeNovo","singleVariant",scores]

In [ ]:
#with open('ULB_Control_vs_NeuroDev.txt', 'w') as outfile:
#    json.dump(scores, outfile)


##Compound

In [ ]:
def scoreCompound(k,variantList):
    variantList=list(variantList)
    result=[]
    if len(variantList)>1:
        for i in range(0,len(variantList)-1):
            (locus1,geno1)=variantList[i]
            for j in range(i+1,len(variantList)):
                (locus2,geno2)=variantList[j]
                (score,listTrios)=getScorePair(geno1,geno2)
                if score>0:
                    result.append(((k,locus1,locus2),score,listTrios,[geno1,geno2]))
        
    return result

In [ ]:
def getScorePair(geno1,geno2):
    score=0
    listTrios=[]
    checkValid=0
    nTrios=len(geno1)/3
    for i in range(0,nTrios):
        checkValid=checkValid+int(int(geno1[i*3+1])>0 and int(geno2[i*3+1])>0)+int(int(geno1[i*3+2])>0 and int(geno2[i*3+2])>0)
        
    if checkValid==0:
        for i in range(0,nTrios):
            scoreTrio=int(geno1[i*3]=="1" and geno2[i*3]=="1" and 
                          ((geno1[i*3+1]=="1" and geno2[i*3+1]=="0" and geno1[i*3+2]=="0" and geno2[i*3+2]=="1") or
                           (geno1[i*3+1]=="0" and geno2[i*3+1]=="1" and geno1[i*3+2]=="1" and geno2[i*3+2]=="0")))
            if scoreTrio>0:
                listTrios.append(i)
                score=score+scoreTrio
    return (score,listTrios)

In [ ]:
#start_time = time.time()
#scores=sc.textFile(filename,nPartitions).map(splitValues).groupByKey().flatMap(lambda (k,v):scoreCompound(k,v)).takeOrdered(100000, key=lambda (k,v1,v2,v3): -v1)
#runtime=time.time() - start_time
#print(runtime)

In [ ]:
#len(scores)

##Digenic

In [ ]:
def scoreDigenic(k,variantLists):
    variantLists=list(variantLists)
    result=[]
    if len(variantLists)==2:
        variantList1=list(variantLists[0])
        variantList2=list(variantLists[1])
        
        for i in range(0,len(variantList1)):
            (gene1,(locus1,geno1))=variantList1[i]
            for j in range(0,len(variantList2)):
                (gene2,(locus2,geno2))=variantList2[j]
                (score,listTrios)=getScorePair(geno1,geno2)
                if score>0:
                    result.append(((gene1,locus1,gene2,locus2),score,listTrios,[geno1,geno2]))
        
    return result

In [ ]:
start_time = time.time()
scores=sc.textFile(filename,nPartitions).map(splitValues).mapPartitionsWithIndex(f).flatMap(lambda (k,v):makePairParts(k,v,nPartitions)).groupByKey().flatMap(lambda (k,v):scoreDigenic(k,v)).takeOrdered(100, key=lambda (k,v1,v2,v3): -v1)
runtime=time.time() - start_time
print(runtime)

In [ ]:
len(scores)

#Gene

##Univariate

In [26]:
def scoreGeneUnivariate(k,variantList):
    variantList=list(variantList)
    result=[k,(0,0,0)]
    sumCase=0
    sumControl=0
    score=0
    genosum=[]
    if len(variantList)>0:
        for i in range(0,len(variantList)):
            (locus,geno)=variantList[i]
            if genosum==[]:
                genosum=[int(x) for x in geno]
            else:
                genosum=[int(x)+int(y) for x,y in zip(genosum,geno)]
                
        sumCase=sum([int(x>0) for x in genosum[0:indexControl]])
        sumControl=sum([int(x>0) for x in genosum[indexControl:len(genosum)]])
        score=sumCase-sumControl
        #if sumControl>0:
        #    score=0

    #if score>0:
    result=[k,(score,sumCase,sumControl)]
    
    return result

In [40]:
scores=sc.textFile(filename,nPartitions).map(splitValues).groupByKey().map(lambda (k,v):scoreGeneUnivariate(k,v)).filter(lambda (k,(v1,v2,v3)): v1>0).takeOrdered(10000, key=lambda (k,(v1,v2,v3)): -v1)

In [41]:
scores[0:10]


[[u'ENSG00000151233', (22, 24, 2)],
 [u'ENSG00000109919', (19, 23, 4)],
 [u'ENSG00000136938', (17, 24, 7)],
 [u'ENSG00000064666', (17, 27, 10)],
 [u'ENSG00000163046', (15, 19, 4)],
 [u'ENSG00000235436', (14, 15, 1)],
 [u'ENSG00000005022', (13, 23, 10)],
 [u'ENSG00000142599', (12, 13, 1)],
 [u'ENSG00000160298', (12, 14, 2)],
 [u'ENSG00000155380', (11, 13, 2)]]

In [42]:
#list(scores.take(2)[1][1])
#scores.take(10)

In [43]:
scores=["tol_ctrl_rare","geneUnivariate",scores]

In [44]:
with open('tol_ctrl_rare.txt', 'w') as outfile:
    json.dump(scores, outfile)

##Bivariate

In [208]:

def scoreDigenicGene(k,variantLists):
#    return (k,variantLists)
    variantLists=list(variantLists)
    result=[]
    geno1sum=[]
    geno2sum=[]
    score=0
    gene1=""
    gene2=""
    sumCase=-1
    sumControl=-1
    if len(variantLists)==2:
        (genes,variantList1)=list(variantLists[0])
        (genes,variantList2)=list(variantLists[1])
        gene1=genes[0]
        gene2=genes[1]
        variantList1=list(variantList1)
        variantList2=list(variantList2)
        for i in range(0,len(variantList1)):
            (locus1,geno1)=variantList1[i]
            if geno1sum==[]:
                geno1sum=[int(x) for x in geno1]
            else:
                geno1sum=[int(x)+int(y) for x,y in zip(geno1sum,b)]
                
        for i in range(0,len(variantList2)):
            (locus2,geno2)=variantList2[i]
            if geno2sum==[]:
                geno2sum=[int(x) for x in geno2]
            else:
                geno2sum=[int(x)+int(y) for x,y in zip(geno2sum,b)]
                
        genosum=[x+y for x,y in zip(geno1sum,geno2sum)]
        sumCase=sum([int(x>0) for x in genosum[0:indexControl]])
        sumControl=sum([int(x>0) for x in genosum[indexControl:len(genosum)]])
        score=sumCase
        if sumControl>0:
            score=0
        
    return (k,((gene1,gene2),score,sumCase,sumControl))



In [209]:
a=[]
b=[0,1]
#sum(b,[0,2])
a = [x+y for x,y in zip(a,b)]
a

[]

In [210]:
def getGene(variantData):
    variantData=variantData.split(" ")
    variantGene=variantData.pop(0)
    
    return (variantGene)


In [211]:
def createPairsGenes(k,v,genes):
#    result=[]
#    for gene in genes:
#        result.append(((k,gene),v))
    return [(str(sorted([k,gene])),(sorted([k,gene]),v)) for gene in genes]


In [212]:
start_time = time.time()
#genes=sc.textFile(filename,nPartitions).map(getGene).distinct().takeOrdered(100000)#.flatMap(lambda (k,v):scoreCompound(k,v)).takeOrdered(100000, key=lambda (k,v1,v2,v3): -v1)
scores=sc.textFile(filename,nPartitions).map(splitValues).groupByKey().flatMap(lambda (k,v):createPairsGenes(k,v,genes)).groupByKey().map(lambda (k,v):scoreDigenicGene(k,v)).takeOrdered(100, key=lambda (k,(genes,v1,v2,v3)): -v1)
runtime=time.time() - start_time
print(runtime)


6.88692593575


In [215]:
scores[0:2]
#scores.take(1)[0][1]
#sorted(['b','a'])


[("[u'ENSG00000186960', u'ENSG00000204120']",
  ((u'ENSG00000186960', u'ENSG00000204120'), 6, 6, 0)),
 ("[u'ENSG00000181626', u'ENSG00000186960']",
  ((u'ENSG00000181626', u'ENSG00000186960'), 5, 5, 0))]

In [216]:
scores=["ULB_Control_vs_NeuroDev_Dam_Gene","geneBivariate",scores]

In [217]:
with open('ULB_Control_vs_NeuroDev_Dam_Gene.txt', 'w') as outfile:
    json.dump(scores, outfile)

#Common results

In [ ]:
#scores

In [ ]:
#scores=["neuroDev_Control","geneUnivariate",scores]

In [ ]:
#with open('neuroDev_Control.txt', 'w') as outfile:
#    json.dump(scores, outfile)

In [ ]:
sc.stop()